In [34]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [14]:
net = cv2.dnn.readNet('../storage/models/yolo_model_1/yolov4-tiny-logistics_size_416_1.weights', 
                      '../storage/models/yolo_model_1/yolov4-tiny-logistics_size_416_1.cfg')

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

with open("../storage/models/yolo_model_1/logistics.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

print(classes)

['barcode', 'car', 'cardboard box', 'fire', 'forklift', 'freight container', 'gloves', 'helmet', 'ladder', 'license plate', 'person', 'qr code', 'road sign', 'safety vest', 'smoke', 'traffic cone', 'traffic light', 'truck', 'van', 'wood pallet']


In [23]:
#sample image
image_path = "../storage/data/dataset/-_57_jpg.rf.6e1bb5dc88e21ab791822b05fcec7d3d.jpg"
image = cv2.imread(image_path)

blob = cv2.dnn.blobFromImage(image,
                              scalefactor = 1/255.,
                              size = (416, 416),
                              mean = (0, 0, 0),
                              swapRB = True,
                              crop = False
)

net.setInput(blob)
outputs = net.forward(output_layers)

In [31]:
for feature_map in outputs:
    for detection in feature_map:
        boxes = detection[:4]
        score = detection[4]
        class_score = detection[5:]